In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [4]:
# Data directory
datadir = "./"

# Load data
data_tensor = torch.load(f"{datadir}/data_test.pt")  # Shape: (1_000_000, 7)

# Split into inputs (first 5 columns) and outputs (last 2 columns)
X_data = data_tensor[:, :5]  # Shape: (1_000_000, 5)
Y_data= data_tensor[:, 5:]  # Shape: (1_000_000, 2)

In [21]:
X_data.shape,Y_data.shape

(torch.Size([10000, 5]), torch.Size([10000, 2]))

In [23]:
Y_data

tensor([[14.7961,  2.9627],
        [ 6.5094,  4.6870],
        [23.4530,  7.9672],
        ...,
        [44.0922,  9.3547],
        [14.0986,  5.5390],
        [24.4505,  7.8674]])

In [24]:
tsun = 4.92549095e-6  # seconds

def true_system(t, y, Mc, eta):
    omega, phi = y
    tN_omega = Mc * omega * tsun
    d_omega = (96 / 5) * omega**2 * tN_omega**(5 / 3)
    d_phi = omega
    return [d_omega, d_phi]

In [12]:
# ==== Neural Network ====
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)  # outputs omega and phi
        )

    def forward(self, x):
        return self.net(x)

In [14]:
def physics_loss(model, X_batch):
    X_batch = X_batch.clone().detach().requires_grad_(True)

    y_pred = model(X_batch)
    omega_pred = y_pred[:, 0:1]
    phi_pred = y_pred[:, 1:2]

    t = X_batch[:, 0:1]
    Mc = X_batch[:, 1:2]

    tsun = 4.92549095e-6
    tN_omega = Mc * omega_pred * tsun

    d_omega = torch.autograd.grad(
        outputs=omega_pred,
        inputs=X_batch,
        grad_outputs=torch.ones_like(omega_pred),
        create_graph=True
    )[0][:, 0:1]  # ∂ω/∂t

    d_phi = torch.autograd.grad(
        outputs=phi_pred,
        inputs=X_batch,
        grad_outputs=torch.ones_like(phi_pred),
        create_graph=True
    )[0][:, 0:1]  # ∂φ/∂t

    res1 = d_omega - (96 / 5) * omega_pred**2 * tN_omega**(5 / 3)
    res2 = d_phi - omega_pred

    loss = (res1 ** 2).mean() + (res2 ** 2).mean()
    return loss

In [15]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [19]:
X_tensor = X_data.clone().detach().float().to(device) if isinstance(X_data, torch.Tensor) \
    else torch.tensor(X_data, dtype=torch.float32).to(device)
dataset = TensorDataset(X_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

model = PINN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [20]:
epochs = 2000
for epoch in range(epochs):
    total_loss = 0
    for (X_batch,) in loader:
        optimizer.zero_grad()
        loss = physics_loss(model, X_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if epoch % 200 == 0:
        print(f"Epoch {epoch}: loss = {total_loss/len(loader):.6f}")

Epoch 0: loss = nan


KeyboardInterrupt: 